In [ ]:
import os
import pandas as pd
import polars as pl
import numpy as np
import kaggle_evaluation.jane_street_inference_server

In [ ]:
file_path = "/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet/date_id=0/part-0.parquet"
df = pl.scan_parquet(file_path)
temp_test = df.collect()  # Collect into a Polars DataFrame

In [ ]:
temp_test = temp_test.to_pandas().fillna(0)
before_test = temp_test.copy()

In [ ]:
# if the sign is same as before, use the before sign
# if the sign is not the same as before, use the opposite previous sign

sign_compute_lambda = lambda row: (
    np.sign(row['feature_01_before']) * 0.078343086
    if np.sign(row['feature_01_now']) == np.sign(row['feature_01_before'])
    else -np.sign(row['feature_01_before']) * 0.078343086
)

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame:
    global before_test
        
    now_test = test.to_pandas().fillna(0).replace({-np.inf: -1, np.inf: 1})
    features_names = ["symbol_id", "feature_01"]
    
    # Apply the lambda function to compute predictions
    merged_df = pd.merge(now_test[features_names], before_test[features_names], on = "symbol_id", how="left", suffixes=('_now','_before'))
    pred = merged_df.apply(sign_compute_lambda, axis=1).fillna(0).values
    
    predictions = test.select('row_id').with_columns(
        pl.Series(
            name='responder_6', 
            values=pred,  
            dtype=pl.Float64
        )
    )
    
    # save the current test a before test, so we can keep it for next round
    before_test = now_test.copy()
    assert len(predictions) == len(test)

    return predictions


In [ ]:
%%time
# Setup the inference server
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

# Running the inference server
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway((
        '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
        '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
    ))